In [2]:
import pandas as pd
import time, threading
# 0 - Revoke, 1 - Post, 2 - Match

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
'''
# CHANGE THIS
WORKING_DIR = r"D:\Data\MOEX-FX\2018-03\\"
OrderLog = "OrderLog20180302"
TradeLog = "TradeLog20180302"
order_log = pd.read_csv(WORKING_DIR + OrderLog + ".txt")
trade_log = pd.read_csv(WORKING_DIR + TradeLog + ".txt")
'''
OrderLog = "OrderLog20180302"
import gzip 
with gzip.open('/content/drive/MyDrive/DataMining/2018-03/OrderLog20180301.txt.gz', 'rb') as f:        
  order_log = pd.read_csv(f)
with gzip.open('/content/drive/MyDrive/DataMining/2018-03/TradeLog20180301.txt.gz', 'rb') as f:        
  trade_log = pd.read_csv(f)

In [5]:
class Action():
    REVOKE = 0
    POST = 1
    MATCH = 2

In [6]:
CODES = ['USD000000TOD', 'USD000UTSTOM', 'EUR_RUB__TOD', 'EUR_RUB__TOM', 'EURUSD000TOD', 'EURUSD000TOM',]

In [7]:
instruments_info = {'USD000000TOD': {'SCHEDULE': 174500000000, 'PRICE_STEP': 0.0025, 'INDEX':0},
                    'USD000UTSTOM': {'SCHEDULE': 235000000000, 'PRICE_STEP': 0.0025, 'INDEX':1},
                    'EUR_RUB__TOD': {'SCHEDULE': 150000000000, 'PRICE_STEP': 0.0025, 'INDEX':2},
                    'EUR_RUB__TOM': {'SCHEDULE': 235000000000, 'PRICE_STEP': 0.0025, 'INDEX':3},
                    'EURUSD000TOM': {'SCHEDULE': 235000000000, 'PRICE_STEP': 0.00001, 'INDEX':4},
                    'EURUSD000TOD': {'SCHEDULE': 150000000000, 'PRICE_STEP': 0.00001, 'INDEX':5}}

In [8]:
# filtering orders
df_order = order_log[(order_log['SECCODE'].isin(CODES)) & (order_log['TIME'] < 235000000000)].copy()[:20000]
#df_order = order_log[(order_log['SECCODE'].isin(CODES)) & (order_log['TIME'] < 100000210798)].copy()[:10000]
df_trade = trade_log[trade_log['SECCODE'].isin(CODES)].copy()

# harvard professor wrote this +100000 IQ code
prep_dic = {Action.POST: 0, Action.MATCH: 1, Action.REVOKE: 2}
unprep_dic = {0: Action.POST, 1: Action.MATCH, 2: Action.REVOKE}

df_order['ACTION'] = df_order['ACTION'].apply(lambda num: prep_dic[num])
df_order.sort_values(by = ['TIME', 'ACTION'], inplace=True)
df_order['ACTION'] = df_order['ACTION'].apply(lambda num: unprep_dic[num])

In [9]:
df_order.head()

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,EUR_RUB__TOD,B,100000050299,1,1,68.610,1000000,NaN,NaN
1,2,USD000000TOD,S,100000050316,2,1,56.600,1000000,NaN,NaN
2,3,EUR_RUB__TOD,S,100000050325,3,1,69.065,1000000,NaN,NaN
3,4,EUR_RUB__TOM,B,100000050353,4,1,68.725,1000000,NaN,NaN
4,5,USD000000TOD,B,100000050361,5,1,56.210,1000000,NaN,NaN


In [10]:
df_order[df_order['NO'] == 100]

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
99,100,EUR_RUB__TOM,S,100000210798,57,0,68.875,1000000,NaN,NaN


In [11]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 25892
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NO          20000 non-null  int64  
 1   SECCODE     20000 non-null  object 
 2   BUYSELL     20000 non-null  object 
 3   TIME        20000 non-null  int64  
 4   ORDERNO     20000 non-null  int64  
 5   ACTION      20000 non-null  int64  
 6   PRICE       20000 non-null  float64
 7   VOLUME      20000 non-null  int64  
 8   TRADENO     746 non-null    float64
 9   TRADEPRICE  746 non-null    float64
dtypes: float64(3), int64(5), object(2)
memory usage: 1.7+ MB


In [12]:
df_trade.head()

,TRADENO,SECCODE,TIME,BUYORDERNO,SELLORDERNO,PRICE,VOLUME
0,163897149,USD000000TOD,100000,88,90,56.3800,70000
1,163897150,USD000UTSTOM,100001,296,628,56.4000,1000
2,163897151,USD000UTSTOM,100001,296,663,56.4000,1000
3,163897152,USD000UTSTOM,100001,701,184,56.4475,1000
4,163897153,USD000UTSTOM,100001,296,881,56.4000,118000


In [13]:
df_trade.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59713 entries, 0 to 61879
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   TRADENO      59713 non-null  int64  
 1   SECCODE      59713 non-null  object 
 2   TIME         59713 non-null  int64  
 3   BUYORDERNO   59713 non-null  int64  
 4   SELLORDERNO  59713 non-null  int64  
 5   PRICE        59713 non-null  float64
 6   VOLUME       59713 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 3.6+ MB


In [14]:
df_order['TRADENO'].value_counts()

163897172.0    2
163897165.0    2
163897268.0    2
163897540.0    2
163897327.0    2
              ..
163897299.0    2
163897158.0    2
163897399.0    2
163897217.0    2
163897458.0    2
Name: TRADENO, Length: 373, dtype: int64

In [15]:
total_mistakes = 0

def print_error(error):
    global total_mistakes
    print('-' * 40)
    print(f'in row: {row["NO"]}')
    print(error)
    print('-' * 40)
    print()
    total_mistakes += 1
    
def check_exists(df_cur, row, NO='ORDERNO'):
    # check if the given ORDERNO exists
    sub_df = df_cur[df_cur[NO] == row[NO]] 
    if len(sub_df) > 0:
        return True, sub_df
    # printing the error
    else:
        print_error(f"ERROR: Record with {NO} {row[NO]} doesn't exist")
        return False, None
    
    
def handle_revoke(df_cur, row):
    # check if the given ORDERNO exists
    check, sub_df = check_exists(df_cur, row)
    if check:
        series = sub_df.iloc[0]
        # check if revoking volume is not greater than the current one
        if series['VOLUME'] >= row['VOLUME']:                
            if series['VOLUME'] == row['VOLUME']:
                # removing the record
                df_cur.drop(df_cur[df_cur['ORDERNO'] == row['ORDERNO']].index, inplace=True)
            else:
                # reducing the amount
                df_cur.loc[df[df_cur['ORDERNO'] == row['ORDERNO']].index, ['VOLUME']] = series['VOLUME'] - row['VOLUME']
        # printing the error
        else:
            # TODO: DISCRUSS WITH THE TEAM
            df_cur.loc[df[df_cur['ORDERNO'] == row['ORDERNO']].index, ['VOLUME']] = series['VOLUME'] - row['VOLUME'] # negative number
            print_error("ERROR: Cannot revoke more that there is")
        
        
def handle_match(df_cur, row):
    global df_trade
    # check if the given ORDERNO exists
    check, sub_df = check_exists(df_cur, row)
    if check:
        series = sub_df.iloc[0]
        # check if the given tradeno exists in trade logs
        check, sub_trade = check_exists(df_trade, row, 'TRADENO')       
        if check:
            series_trade = sub_trade.iloc[0]

            seller = df_cur[df_cur['ORDERNO'] == series_trade['SELLORDERNO']]
            buyer = df_cur[df_cur['ORDERNO'] == series_trade['BUYORDERNO']]

            # check if seller and buyer exist
            if len(seller) > 0 and len(buyer) > 0:
                seller = seller.iloc[0]
                buyer = buyer.iloc[0]

                # check if the volume is ok
                if seller['VOLUME'] >= row['VOLUME'] and buyer['VOLUME'] >= row['VOLUME']:
                    if seller['VOLUME'] == row['VOLUME']:
                        # removing the record
                        df_cur.drop(df_cur[df_cur['ORDERNO'] == seller['ORDERNO']].index, inplace=True)
                    else:
                        # reducing the amount
                        df_cur.loc[df_cur['ORDERNO'] == seller['ORDERNO'], ['VOLUME']] = seller['VOLUME'] - row['VOLUME']

                    if buyer['VOLUME'] == row['VOLUME']:
                        # removing the record
                        df_cur.drop(df_cur[df_cur['ORDERNO'] == buyer['ORDERNO']].index, inplace=True)
                    else:
                        # reducing the amount
                        df_cur.loc[df_cur['ORDERNO'] == buyer['ORDERNO'], ['VOLUME']] = buyer['VOLUME'] - row['VOLUME']

                # printing the error
                else:
                    # TODO: Discuss with the team
                    # negative numbers
                    df_cur.loc[df_cur['ORDERNO'] == seller['ORDERNO'], ['VOLUME']] = seller['VOLUME'] - row['VOLUME']
                    df_cur.loc[df_cur['ORDERNO'] == buyer['ORDERNO'], ['VOLUME']] = buyer['VOLUME'] - row['VOLUME']
                    print_error("ERROR: Not enough amount for buying and/or selling")

            # printing the error
            else:
                print_error(f"ERROR: There's no such buyer and/or seller")               


In [16]:
df_order[df_order['PRICE'] == 0]

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
861,862,USD000UTSTOM,B,100001279110,701,1,0.0,1000,NaN,NaN
862,863,USD000UTSTOM,B,100001279110,701,2,0.0,1000,163897152.0,56.4475
1151,1152,USD000UTSTOM,B,100001597883,937,1,0.0,130000,NaN,NaN
1152,1153,USD000UTSTOM,B,100001597883,937,2,0.0,81000,163897155.0,56.4000
1154,1155,USD000UTSTOM,B,100001597883,937,2,0.0,49000,163897156.0,56.4275
...,...,...,...,...,...,...,...,...,...,...
22792,22793,USD000000TOD,S,100110668627,11883,2,0.0,15000,163897479.0,56.3800
23832,23833,EUR_RUB__TOM,S,100114045815,12401,1,0.0,100000,NaN,NaN
23834,23835,EUR_RUB__TOM,S,100114045815,12401,2,0.0,100000,163897494.0,68.7875
24351,24352,USD000000TOD,S,100117063156,12655,1,0.0,24000,NaN,NaN


In [23]:
# ГОВНОКОД NO.1
# блин хз возможно класс больше не нужен

class Spectrum():
  def __init__(self, seccode):
        self.seccode = seccode 
        self.best_ask = 100000000
        self.best_bid = -1
        self.bids = [0]*10
        self.asks = [0]*10
'''
оказалось ненужным :(
  def calculate_steps(self, step):
        b = self.best_bid
        a = self.best_ask
        bids = []
        asks = []
        asks.append(a)
        bids.append(b)
        a = a-step
        b=b+step
        for i in range(10):
          a = round(a+5*step, 5)
          asks.append(a)
          b = round(b-5*step, 5)
          bids.append(b)
        bids.reverse()
        return bids, asks
'''

'\nоказалось ненужным :(\n  def calculate_steps(self, step):\n        b = self.best_bid\n        a = self.best_ask\n        bids = []\n        asks = []\n        asks.append(a)\n        bids.append(b)\n        a = a-step\n        b=b+step\n        for i in range(10):\n          a = round(a+5*step, 5)\n          asks.append(a)\n          b = round(b-5*step, 5)\n          bids.append(b)\n        bids.reverse()\n        return bids, asks\n'

In [24]:
# ГОВНОКОД NO.2

spectrums = []
for i in instruments_info.keys():
  spectrums.append(Spectrum(i))

In [39]:
# ГОВНОКОД NO.3

import math

def change_spectrums(df_cur, row):   #seccode = row['SECCODE']
    seccode = row['SECCODE']
    step = instruments_info[seccode]['PRICE_STEP']
    #if row['BUYSELL'] == 'B':
    #  if spectrums[instruments_info[seccode]['INDEX']].best_bid < row['PRICE']:
    spectrum = spectrums[instruments_info[seccode]['INDEX']]
    
    new_price = row['PRICE']

    if row['ACTION'] == Action.POST:

      if row['BUYSELL'] == 'B':
          if new_price > spectrum.best_bid:
            temp_b = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'B')]
            spectrum.best_bid = new_price
            new_bids = [0]*10
            for index, row in temp_b.iterrows():
              price = row['PRICE']
              dif = spectrum.best_bid - price
              if dif > step*49:
                pass
              else:
                new_bids[9-math.floor(dif/(step*5))] += row['VOLUME']
            #spectrum.bids = new_bids.reverse()
          elif new_price == spectrum.best_bid:
            new_bids = spectrum.bids
            new_bids[9]+= row['VOLUME']
            spectrum.bids = new_bids
          else:
            dif = spectrum.best_bid-new_price 
            if dif > step*49:
              pass
            else:
              spectrum.bids[9-math.floor(dif/(step*5))] += row['VOLUME']

      if row['BUYSELL'] == 'S':
          if new_price < spectrum.best_ask:
            temp_s = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'S')]
            spectrum.best_ask = new_price
            new_asks = [0]*10
            for index, row in temp_s.iterrows():
              price = row['PRICE']
              dif = price-spectrum.best_ask 
              if dif > step*49:
                pass
              else:
                new_asks[math.floor(dif/(step*5))] += row['VOLUME']
            spectrum.asks = new_asks
          elif new_price == spectrum.best_ask:
            new_asks = spectrum.asks
            new_asks[0] += row['VOLUME']
            spectrum.asks = new_asks 
          else:
            dif = new_price-spectrum.best_ask 
            if dif > step*49:
              pass
            else:
              spectrum.asks[math.floor(dif/(step*5))] += row['VOLUME']
    
    elif row['ACTION'] == Action.REVOKE:
      if row['BUYSELL'] == 'B':
        if new_price > spectrum.best_bid:
          pass
        else:
          dif = spectrum.best_bid - new_price
          if dif > step*49:
            pass
          else:
            new_bids = spectrum.bids
            new_bids[9-math.floor(dif/(step*5))] -= row['VOLUME']
            spectrum.bids = new_bids
      if row['BUYSELL'] == 'S':
        if new_price < spectrum.best_ask:
          pass
        else:
          dif = new_price-spectrum.best_ask 
          if dif > step*49:
            pass
          else:
            new_asks = spectrum.asks
            new_asks[math.floor(dif/(step*5))] -= row['VOLUME']
            spectrum.asks = new_asks

    b = spectrum.bids
    values = b.append(spectrum.asks)
    return values

'''
    temp_b = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'B')]
    temp_s = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'S')]
    spectrum.best_bid = temp_b['PRICE'].max()
    spectrum.best_ask = temp_s['PRICE'].min()
    #bids, asks = spectrum.calculate_steps(step)
    values = [0] * 20
    for index, row in temp_s.iterrows():
      price = row['PRICE']
      dif = price - spectrum.best_ask
      if dif > step*49:
        pass
      else:
        values[10+math.floor(dif/(step*5))] += row['VOLUME']
    for index, row in temp_b.iterrows():
      price = row['PRICE']
      dif = spectrum.best_bid - price
      if dif > step*49:
        pass
      else:
        values[math.floor(dif/(step*5))] += row['VOLUME']
    return values
'''  



"\n    temp_b = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'B')]\n    temp_s = df_cur.loc[(df_cur['SECCODE'] == seccode) & (df_cur['BUYSELL'] == 'S')]\n    spectrum.best_bid = temp_b['PRICE'].max()\n    spectrum.best_ask = temp_s['PRICE'].min()\n    #bids, asks = spectrum.calculate_steps(step)\n    values = [0] * 20\n    for index, row in temp_s.iterrows():\n      price = row['PRICE']\n      dif = price - spectrum.best_ask\n      if dif > step*49:\n        pass\n      else:\n        values[10+math.floor(dif/(step*5))] += row['VOLUME']\n    for index, row in temp_b.iterrows():\n      price = row['PRICE']\n      dif = spectrum.best_bid - price\n      if dif > step*49:\n        pass\n      else:\n        values[math.floor(dif/(step*5))] += row['VOLUME']\n    return values\n"

In [ ]:
start = time.time()
k = 0
# empty df for keeping track
df_cur = pd.DataFrame(columns=df_order.columns)
df_spec = pd.DataFrame(columns=['SECCODE', 'TIMESTAMP', 'BID_ASK'])

# for processing each TRADENO only once
matches = []

# iterating through the order book
for index, row in df_order.iterrows():
    # if post -> add to df_cur
    if row['ACTION'] == Action.POST:
        df_cur = df_cur.append(row)
        
    
    elif row['ACTION'] == Action.REVOKE:
        handle_revoke(df_cur, row)
       
        
    elif row['ACTION'] == Action.MATCH:
        if row['TRADENO'] in matches:
            matches.remove(row['TRADENO'])
            continue
        else:
            matches.append(row['TRADENO'])
            handle_match(df_cur, row)
    
    # для каждой новой row считаем спектрум и добавляем в файлик
    values = change_spectrums(df_cur, row)
    d = {'SECCODE':row['SECCODE'], 'TIMESTAMP':row['TIME'], 'BID_ASK':values}
    df_spec = df_spec.append(d, ignore_index=True)
    k+=1
    if k%50000 == 0:
      df_spec.to_csv('/content/drive/MyDrive/DataMining/spectrums/'+str(k//50000)+'.csv')  


            
            
df_cur.to_csv('no-thread-lol.csv')
df_spec.to_csv('spectru.csv')

end = time.time()

print(end - start)

In [ ]:
df_spec.to_csv('/content/drive/MyDrive/DataMining/spectrum.csv')

In [ ]:
df_spec

In [ ]:
final = df_cur.groupby(['SECCODE','BUYSELL','PRICE'])['VOLUME'].sum().reset_index()\
.sort_values(by=['SECCODE', 'BUYSELL', 'PRICE'])

In [ ]:
final

In [ ]:
final.to_csv('OrderBook' + OrderLog[8:] +'.csv')

In [ ]:
# With multithreading

# df_bycodes = []

# for code in CODES:
#     df_bycodes.append(df_order[df_order['SECCODE'] == code])
    

# def handle_thread(df_order, code, time):
#     # empty df for keeping track
#     df_cur = pd.DataFrame(columns=df_order.columns)

#     prev_match = False
#     prev_time = -1
#     # iterating through the order book
#     for index, row in df_order.iterrows():
#         # if post -> add to df_cur
#         if row['ACTION'] == Action.POST:
#             df_cur = df_cur.append(row)
#             prev_match = False


#         elif row['ACTION'] == Action.REVOKE:
#             handle_revoke(df_cur, row)
#             prev_match = False


#         elif row['ACTION'] == Action.MATCH:
#             if prev_match:
#                 prev_match = False
#                 continue
#             else:
#                 handle_match(df_cur, row)
#                 prev_match = True

#     df_cur.to_csv('thread_' + code + '_' + time + '.csv')

    
# threads = []

# start = time.time()

# for index, code in enumerate(CODES):
#     thread = threading.Thread(target=handle_thread, args=(df_bycodes[index], code, 'TEST_TIME'))
#     threads.append(thread)
#     thread.start()
    
# for thread in threads:
#     thread.join()
    
# end = time.time()

# print(end - start)
    

### Mistakes in dataset

#### Incorrect volumes

In [ ]:
df_order[df_order['NO'] == 95871]

In [ ]:
df_order[df_order['ORDERNO'] == 48103]

In [ ]:
df_trade[df_trade['TRADENO'] == 164004000]

In [ ]:
df_order[df_order['ORDERNO'] == 48095]

#### Вот тут думаю надо исправить (кривой порядок NO в одном TIME)

#### upd: temporarly solved

In [ ]:
df_order[df_order['NO'] == 95877]

In [ ]:
df_order[df_order['ORDERNO'] == 48104]

In [ ]:
df_trade[df_trade['TRADENO'] == 164004001]

In [ ]:
df_order[df_order['ORDERNO'] == 48105]

In [ ]:
df_order[(df_order['NO'] >= 95876) & (df_order['NO'] <= 95880)]

In [ ]:
df_order[(df_order['NO'] >= 95876) & (df_order['NO'] <= 95880)]